In [1]:
import pandas as pd
import os
import shutil
import json
import ast
import re
import tqdm
from typing import List, Dict

In [ ]:
swapped_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed"
swapped_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed"

orig_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk"
orig_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng"

In [ ]:
def parse_ann_file(ann_file):
    entities = []

    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                parts = line.strip().split("\t")
                if len(parts) < 3:
                    continue
                label = parts[1]
                start = parts[2]
                end = parts[3]
                text = parts[4]

                entities.append({"ind": parts[0], "text": text, "start": start, "end": end, "label": label})
            except:
                continue

    return entities


def extract_entities(txt_file: str, ann_file: str):
    labeled_sentences_list = []
    sentences = []

    with open(txt_file, "r", encoding="utf-8") as f:
        sentence_lines = f.readlines()

    entities = parse_ann_file(ann_file)
    empty_lines_count = sentence_lines.count("\n")
    
    actual_counter = -1
    used_tags = []
    sent_end = 0
    for ind, sentence in enumerate(sentence_lines):
        actual_counter += 1
        sent_end += len(sentence)
        labeled_dict = {}
        job = False

        if sentence == "\n":
            actual_counter -= 1
        sentence = sentence.strip()

        for ent in entities:
            if re.search(rf"\b{re.escape(ent['text'])}\b", sentence):
                if ent["label"] == "JOB" and ent['ind'] not in used_tags and int(ent['start']) < sent_end:
                    labeled_dict[ent['ind']] = (ent["text"], ent["label"])
                    used_tags.append(ent['ind'])
                    job = True

        if job:
            labeled_sentences_list.append(labeled_dict)
            sentences.append((ind, sentence, actual_counter))

    return labeled_sentences_list, sentences, empty_lines_count, len(sentence_lines)


def get_sentences_with_job_titles(txt_file_names: List[str]):
    sentence_with_jobs = {}

    for txt_file in txt_file_names:
        ann_file = txt_file.replace(".txt", ".ann")
        labeled_sentences_list, sentences, empty_lines_count, numb_of_sents = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i][1]] = (sentences[i][0], labeled_sentences_list[i],  empty_lines_count, numb_of_sents, sentences[i][2])

    return sentence_with_jobs


def process_files(original_folder: str, changed_folder: str):
    dataset = []

    original_files = [f for f in os.listdir(original_folder) if f.endswith('.txt')]

    for filename in tqdm.tqdm(original_files):
        orig_txt_path = os.path.join(original_folder, filename)
        changed_txt_path = os.path.join(changed_folder, filename.replace('.txt', '_1.txt'))

        if os.path.exists(changed_txt_path):

            job_sentences = get_sentences_with_job_titles([orig_txt_path])

            with open(changed_txt_path, 'r', encoding='utf-8') as f:
                changed_sentences = f.readlines()

            for orig_sent, val in job_sentences.items():
                if len(changed_sentences) != val[3]:
                    changed_sent = changed_sentences[val[4]]
                    dataset.append([orig_sent, val[0], changed_sent, val[4], filename, val[1]])
                else:
                    changed_sent = changed_sentences[val[0]]
                    dataset.append([orig_sent, val[0], changed_sent.strip(), val[0], filename, val[1]])

    columns = ["original_sentence", "orig_sent_id", "changed_sentence", "changed_sent_id", "original_file_name", "orig_ann"]
    return pd.DataFrame(dataset, columns=columns)

In [ ]:
parallel_dataset_bruk_dataset = process_files(orig_bruk_folder, swapped_bruk_folder)

In [ ]:
parallel_dataset_ng_dataset = process_files(orig_ng_folder, swapped_ng_folder)

In [ ]:
def find_changed_ann(orig_annotation, filename, path):
    
    list_of_inds = [ind for ind, ent in orig_annotation.items()]

    changed_ann_file = path + filename.replace(".txt", "_1.ann")

    entities = parse_ann_file(changed_ann_file)
    labeled_dict = {}

    for ent in entities:
           if ent['ind'] in list_of_inds:
                labeled_dict[ent["ind"]] = (ent['text'], ent["label"])
    return labeled_dict

parallel_dataset_bruk_dataset['changed_ann'] = parallel_dataset_bruk_dataset.apply(lambda x: find_changed_ann(x['orig_ann'], x['original_file_name'], "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/bruk_changed/"), axis=1)
parallel_dataset_ng_dataset['changed_ann'] = parallel_dataset_ng_dataset.apply(lambda x: find_changed_ann(x['orig_ann'], x['original_file_name'], "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/ng_changed/"), axis=1)

In [4]:
parallel_dataset_bruk_dataset.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
0,Його редактором був поет-символіст Яків Савчен...,46,Його редакторкою була поетеса-символістка Анже...,46,e5e76a8efa0f.txt,"{'T30': ('поет-символіст', 'JOB')}","{'T30': ('поетеса-символістка', 'JOB')}"
1,Міжпредметні паралелі . Маніфест футуристів ск...,62,Міжпредметні паралелі . Маніфест футуристок ск...,62,e5e76a8efa0f.txt,"{'T49': ('поет', 'JOB')}","{'T49': ('поетеса', 'JOB')}"
2,Помітною була організація « Гарт » ( 1923 — 19...,96,Помітною була організація « Гарт » ( 1923 — 19...,96,e5e76a8efa0f.txt,"{'T104': ('поет', 'JOB')}","{'T104': ('поетка', 'JOB')}"
3,Її очолив байкар і прозаїк Сергій Пилипенко .,127,Її очолила байкарка і прозаїкиня Марія Пилипен...,127,e5e76a8efa0f.txt,"{'T143': ('байкар', 'JOB'), 'T144': ('прозаїк'...","{'T143': ('байкарка', 'JOB'), 'T144': ('прозаї..."
4,У Галицько-Волинському літописі згадується спі...,171,У Галицько-Волинському літописі згадується спі...,171,e5e76a8efa0f.txt,"{'T218': ('співець', 'JOB')}","{'T218': ('співчиня', 'JOB')}"


In [5]:
parallel_dataset_ng_dataset.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
0,"Слідчі встановили, що у 2018-2019 рр. управлін...",2,"Слідчі встановили, що у 2018-2019 рр. управлін...",2,104263660695.txt,"{'T6': ('Слідчі', 'JOB')}","{'T6': ('Слідчі', 'JOB')}"
1,Суд надав слідчим доступ до документації щодо ...,7,Суд надав слідчій доступ до документації щодо ...,7,104263660695.txt,"{'T23': ('слідчим', 'JOB')}","{'T23': ('слідчій', 'JOB')}"
2,"Директором та засновником ТОВ «Армшляхбуд», ст...",8,"Директоркою та засновницею ТОВ «Армшляхбуд», с...",8,104263660695.txt,"{'T25': ('Директором', 'JOB'), 'T26': ('заснов...","{'T25': ('Директоркою', 'JOB'), 'T26': ('засно..."
3,Співвласником цієї компанії у свою чергу з час...,5,Співвласницею цієї компанії у свою чергу з час...,5,97c7987b8f9c.txt,"{'T17': ('голова наглядової ради', 'JOB')}","{'T17': ('голова наглядової ради', 'JOB')}"
4,Одесит Іван Плачков почав працювати в компанії...,6,Одеситка Іванна Плачкова почала працювати в ко...,6,97c7987b8f9c.txt,"{'T25': ('головою правління', 'JOB')}","{'T25': ('головою правління', 'JOB')}"


In [ ]:
parallel_dataset_bruk_dataset.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv", index=0)
parallel_dataset_ng_dataset.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv", index=0)